In [13]:
from sklearn.ensemble import RandomForestClassifier as RandomForest
from sklearn.linear_model import LogisticRegression
import sys
import numpy as np
import torch
#sys.path.append ('/Users/tanvikapoor/zr-obp')
#print(sys.path)

# import open bandit pipeline (obp)
import obp
from obp.dataset import (
    SyntheticBanditDataset,
    logistic_reward_function,
    linear_reward_function
)
from obp.policy import (
    IPWLearner, 
    QLearner,
    NNPolicyLearner, 
    Random
)

import loss_translation

## (1) Generating Synthetic Data
`obp.dataset.SyntheticBanditDataset` is an easy-to-use synthetic data generator.

It takes 
- number of actions (`n_actions`, $|\mathcal{A}|$)
- dimension of context vectors (`dim_context`, $d$)
- reward function (`reward_function`, $q(x,a)=\mathbb{E}[r|x,a]$)

as inputs and generates synthetic logged bandit data that can be used to evaluate the performance of decision making policies (obtained by `off-policy learning`).

In [14]:
# generate synthetic logged bandit data with 10 actions
# we use `logistic function` as the reward function and control the behavior policy with `beta`
# one can define their own reward function and behavior policy function such as nonlinear ones. 
dataset = SyntheticBanditDataset(
    n_actions=10,
    dim_context=5,
    beta=-2, # inverse temperature parameter to control the optimality and entropy of the behavior policy
    reward_type="binary", # "binary" or "continuous"
    reward_function=logistic_reward_function,
    random_state=12345,
)

In [15]:
# obtain training and test sets of synthetic logged bandit data
n_rounds_train, n_rounds_test = 10000, 10000
bandit_feedback_train = dataset.obtain_batch_bandit_feedback(n_rounds=int(0.8*n_rounds_train))
bandit_feedback_val = dataset.obtain_batch_bandit_feedback(n_rounds=int(0.2*n_rounds_train))
bandit_feedback_test = dataset.obtain_batch_bandit_feedback(n_rounds=n_rounds_test)

## (2) Off-Policy Learning
After generating synthetic data, we now train some decision making policies.

To train policies on logged bandit data, we use

- `obp.policy.NNPolicyLearner` (Neural Network Policy Learner)
- `obp.policy.IPWLearner`

For `NN Learner`, we use 
- Direct Method ("dm")
- InverseProbabilityWeighting ("ipw")
- DoublyRobust ("dr") 

as its objective functions (`off_policy_objective`). 

For `IPW Learner`, we use `RandomForestClassifier` and *LogisticRegression* implemented in scikit-learn for base ML methods.

A policy is trained by maximizing an OPE estimator as an objective function as follows.

$$ \hat{\pi} \in \arg \max_{\pi \in \Pi} \hat{V} (\pi; \mathcal{D}_{tr}) - \lambda \cdot \Omega (\pi)  $$

where $\hat{V}(\cdot; \mathcal{D})$ is an off-policy objective and $\mathcal{D}_{tr}$ is a training bandit dataset. $\Omega (\cdot)$ is a regularization term.

In [16]:
max_iters = [200, 400, 600]
random_states = [12345, 54321, 11111]

with open("evaluation.txt", "a") as o:

  for m in max_iters:
    for r in random_states:

        print(dataset)
        
        den_l = loss_translation.binarySearch (m, r, 0, 0.0001, dataset, bandit_feedback_train, 0, 100, 0.01, 1, 0.05)
        
        def black_box_function(l):
        #l: Hyperparameter to optimize for, which is l
          f = loss_translation.predict_value_ratio_train (m, r, 0, 0.0001, dataset, l, bandit_feedback_train)
          return f
          
        pbounds = {"l": [0, 1]}
        init_pnts = 15
        num_iter = 20
        (ratio_train_l, ratio_train_val) = loss_translation.bayesOpt(black_box_function, pbounds, init_pnts, num_iter)

        def black_box_function(l):
        # l: Hyperparameter to optimize for, which is l
          f = loss_translation.predict_value_ratio_val (m, r , 0, 0.0001, dataset, l, bandit_feedback_train, bandit_feedback_val)
          return f

        pbounds = {"l": [0, 1]}
        init_pnts = 15
        num_iter = 20
        (ratio_val_l, ratio_val_val) = loss_translation.bayesOpt(black_box_function, pbounds, init_pnts, num_iter)

        # Policy with no loss
        nn_ipw = loss_translation.generateModel(dataset, m,0,r, 0, 0.0001)

        nn_ipw.fit(
              context=bandit_feedback_train["context"],
              action=bandit_feedback_train["action"],
              reward=bandit_feedback_train["reward"],
              pscore=bandit_feedback_train["pscore"],
            )

        action_dist_nn_ipw_no_loss = nn_ipw.predict_proba(
              context=bandit_feedback_test["context"]
            )

      # Policy with denominator = 1 for training data
        nn_ipw = loss_translation.generateModel(dataset, m,den_l,r, 0, 0.0001)

        nn_ipw.fit(
              context=bandit_feedback_train["context"],
              action=bandit_feedback_train["action"],
              reward=bandit_feedback_train["reward"],
              pscore=bandit_feedback_train["pscore"],
            )

        action_dist_nn_ipw_den_1 = nn_ipw.predict_proba(
              context=bandit_feedback_test["context"]
            )

      # Policy with max ratio for training data
        nn_ipw = loss_translation.generateModel(dataset, m,ratio_train_l ["l"],r, 0, 0.0001)

        nn_ipw.fit(
              context=bandit_feedback_train["context"],
              action=bandit_feedback_train["action"],
              reward=bandit_feedback_train["reward"],
              pscore=bandit_feedback_train["pscore"],
            )

        action_dist_nn_ipw_max_train_ratio = nn_ipw.predict_proba(
              context=bandit_feedback_test["context"]
          )

      # Policy with max ratio for validation data
        nn_ipw = loss_translation.generateModel(dataset, m,ratio_val_l ["l"],r, 0, 0.0001)

        nn_ipw.fit(
              context=bandit_feedback_train["context"],
              action=bandit_feedback_train["action"],
              reward=bandit_feedback_train["reward"],
              pscore=bandit_feedback_train["pscore"],
            )

        action_dist_nn_ipw_max_val_ratio = nn_ipw.predict_proba(
              context=bandit_feedback_test["context"]
          )

        policy_names = [
        "NN Policy Learner with no loss",
        "NN Policy Learner with training denominator = 1",
        "NN Policy Learner with max training ratio",
        "NN Policy Learner with max validation ratio"
          ]
        action_dist_list = [
          action_dist_nn_ipw_no_loss,
          action_dist_nn_ipw_den_1,
          action_dist_nn_ipw_max_train_ratio,
          action_dist_nn_ipw_max_val_ratio
          ]

        with open("evaluation.txt", "a") as o:

          o.write("With param values max_iter: " + str(m) +" and random state: " + str(r) + "\n")

          for name, action_dist in zip(policy_names, action_dist_list):
            true_policy_value = dataset.calc_ground_truth_policy_value(
            expected_reward=bandit_feedback_test["expected_reward"],
            action_dist=action_dist,
            )
            o.write(f'policy value of {name}: {true_policy_value}')

o.close()

policy learning:   0%|          | 0/200 [00:00<?, ?it/s]

SyntheticBanditDataset(n_actions=10, dim_context=5, reward_type='binary', reward_function=<function logistic_reward_function at 0x7fdd524998b0>, reward_std=1.0, action_context=array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]), behavior_policy_function=None, beta=-2, n_deficient_actions=0, random_state=12345, dataset_name='synthetic_bandit_dataset')
[0 1 0 ... 0 0 0]
[-0.5  0.5 -0.5 ... -0.5 -0.5 -0.5]
Here


policy learning:   0%|          | 0/200 [00:00<?, ?it/s]

0.771697911507026
[0 1 0 ... 0 0 0]
[-0.5  0.5 -0.5 ... -0.5 -0.5 -0.5]
Here


policy learning:   0%|          | 1/200 [00:00<00:38,  5.13it/s]

...

policy learning:   0%|          | 0/400 [00:00<?, ?it/s]

[0 1 0 ... 0 0 0]
[-0.5  0.5 -0.5 ... -0.5 -0.5 -0.5]
Here


policy learning:   0%|          | 0/400 [00:00<?, ?it/s]

|  25       |  0.8401   |  0.8059   |
[0 1 0 ... 0 0 0]
[-0.5  0.5 -0.5 ... -0.5 -0.5 -0.5]
Here


policy learning:  97%|█████████▋| 388/400 [01:17<00:02,  5.39it/s]

With param values max_iter: 200 and random state: 12345 <br> 

policy value of NN Policy Learner with no loss: 0.7386491310472931<br>
policy value of NN Policy Learner with training denominator = 1: 0.7747569465293875<br>
policy value of NN Policy Learner with max training ratio: 0.7748265861941667<br>
policy value of NN Policy Learner with max validation ratio: 0.7680523661659651<br>

With param values max_iter: 200 and random state: 54321 <br>

policy value of NN Policy Learner with no loss: 0.7426535159127408 <br>
policy value of NN Policy Learner with training denominator = 1: 0.7715344364047754 <br>
policy value of NN Policy Learner with max training ratio: 0.7731315919182911 <br>
policy value of NN Policy Learner with max validation ratio: 0.7732276796432124 <br>

With param values max_iter: 200 and random state: 11111<br>

policy value of NN Policy Learner with no loss: 0.7525011085808299<br>
policy value of NN Policy Learner with training denominator = 1: 0.7720505838029407<br>
policy value of NN Policy Learner with max training ratio: 0.7730628901592789<br>
policy value of NN Policy Learner with max validation ratio: 0.7668315147195144<br>

In [ ]:
policy_reg_param = [0, 0.001]
learning_rate = [0.0001, 0.001]
random_states = [12345, 54321, 11111, 47801, 60737]

with open("validation_evaluation.txt", "a") as o:

  for p in policy_reg_param:
    for lr in learning_rate:
      for r in random_states:

          print(dataset)
          
          den_l = loss_translation.binarySearch (200, r, p, lr, dataset, bandit_feedback_train, 0, 100, 0.01, 1, 0.05)
          
          def black_box_function(l):
          #l: Hyperparameter to optimize for, which is l
            f = loss_translation.predict_value_ratio_train (200, r, p, lr, dataset, l, bandit_feedback_train)
            return f
            
          pbounds = {"l": [0, 1]}
          init_pnts = 15
          num_iter = 20
          (ratio_train_l, ratio_train_val) = loss_translation.bayesOpt(black_box_function, pbounds, init_pnts, num_iter)

          def black_box_function(l):
          # l: Hyperparameter to optimize for, which is l
            f = loss_translation.predict_value_ratio_val (200, r , p, lr, dataset, l, bandit_feedback_train, bandit_feedback_val)
            return f

          pbounds = {"l": [0, 1]}
          init_pnts = 15
          num_iter = 20
          (ratio_val_l, ratio_val_val) = loss_translation.bayesOpt(black_box_function, pbounds, init_pnts, num_iter)

          # Policy with no loss
          nn_ipw = loss_translation.generateModel(dataset, 200, 0, r, p, lr)

          nn_ipw.fit(
                context=bandit_feedback_train["context"],
                action=bandit_feedback_train["action"],
                reward=bandit_feedback_train["reward"],
                pscore=bandit_feedback_train["pscore"],
              )

          action_dist_nn_ipw_no_loss = nn_ipw.predict_proba(
                context=bandit_feedback_val["context"]
              )

        # Policy with denominator = 1 for training data
          nn_ipw = loss_translation.generateModel(dataset, 200, den_l, r, p, lr)

          nn_ipw.fit(
                context=bandit_feedback_train["context"],
                action=bandit_feedback_train["action"],
                reward=bandit_feedback_train["reward"],
                pscore=bandit_feedback_train["pscore"],
              )

          action_dist_nn_ipw_den_1 = nn_ipw.predict_proba(
                context=bandit_feedback_val["context"]
              )

        # Policy with max ratio for training data
          nn_ipw = loss_translation.generateModel(dataset, 200, ratio_train_l ["l"], r, p, lr)

          nn_ipw.fit(
                context=bandit_feedback_train["context"],
                action=bandit_feedback_train["action"],
                reward=bandit_feedback_train["reward"],
                pscore=bandit_feedback_train["pscore"],
              )

          action_dist_nn_ipw_max_train_ratio = nn_ipw.predict_proba(
                context=bandit_feedback_val["context"]
            )

        # Policy with max ratio for validation data
          nn_ipw = loss_translation.generateModel(dataset, 200, ratio_val_l ["l"], r, p, lr)

          nn_ipw.fit(
                context=bandit_feedback_train["context"],
                action=bandit_feedback_train["action"],
                reward=bandit_feedback_train["reward"],
                pscore=bandit_feedback_train["pscore"],
              )

          action_dist_nn_ipw_max_val_ratio = nn_ipw.predict_proba(
                context=bandit_feedback_val["context"]
            )

          policy_names = [
          "NN Policy Learner with no loss",
          "NN Policy Learner with training denominator = 1",
          "NN Policy Learner with max training ratio",
          "NN Policy Learner with max validation ratio"
            ]
          action_dist_list = [
            action_dist_nn_ipw_no_loss,
            action_dist_nn_ipw_den_1,
            action_dist_nn_ipw_max_train_ratio,
            action_dist_nn_ipw_max_val_ratio
            ]

          o.write("With param values policy regularization: " + str(p) +" and learning rate: " + str(lr) + " and random state: " + str(r) + "\n")

          for name, action_dist in zip(policy_names, action_dist_list):
            true_policy_value = dataset.calc_ground_truth_policy_value(
            expected_reward=bandit_feedback_test["expected_reward"],
            action_dist=action_dist,
            )
            o.write(f'policy value of {name}: {true_policy_value}')

o.close()